<a href="https://colab.research.google.com/github/MTahaRF/Manim-Video-Generator/blob/main/GPTManimVideoGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation And Imports

In [1]:
OPENAI_API_KEY = input("Enter your OpenAI API Key: ")
AZURE_OPENAI_ENDPOINT = input("Enter your Azure OpenAI Endpoint: ")
AZURE_API_VERSION = input("Enter your Azure OpenAI API Version: ")

In [2]:
%pip install -qU 'langgraph' 'langchain-openai' 'langchain-community' 'pydub'
!pip install moviepy
!pip install gtts
!apt-get install imagemagick
!sudo apt update
!sudo apt install libcairo2-dev \
    texlive texlive-latex-extra texlive-fonts-extra dvipng\
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install IPython==8.21.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.5 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts imagemagick-6-common
  imagemagick-6.q16 libdjvulibre-text libdjvulibre21 libfftw3-double3 libgs9 libgs9-common libidn12
  libijs-0.35 libjbig2dec0 libjxr-tools libjxr0 liblqr-1-0 libmagickcore-6.q16-6
  libmagickcore-6.q16-6-extra libmagickwand-6.q16-6 libnetpbm10 libwmflite-0.2-7 netpbm
  poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fo

In [4]:
from langchain_openai import ChatOpenAI
from langchain.chat_models import ChatOpenAI
from IPython.display import HTML, Markdown, display
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.chat_models import init_chat_model
import getpass
import os
import re
import requests
from gtts import gTTS

In [5]:
from moviepy.config import change_settings
change_settings({"IMAGEMAGICK_BINARY": "/usr/bin/convert"})

In [ ]:
#Groq_API_KEY = "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"

In [6]:
from manim import *
config.media_embed = True

In [18]:
import os
import re
import subprocess
import shutil
import time
from gtts import gTTS
from moviepy.editor import (VideoFileClip, concatenate_videoclips,
                            concatenate_audioclips, AudioFileClip, AudioClip)
from moviepy.video.fx.freeze import freeze
from manim import *
from langchain_openai import AzureChatOpenAI

# -----------------------------------------------
# Configuration & Initialization
# -----------------------------------------------

# Set up your OpenAI API key
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
AZURE_DEPLOYMENT_NAME = "gpt-4o-mini"
AZURE_API_VERSION = AZURE_API_VERSION

# Initialize AI models
script_model = AzureChatOpenAI(deployment_name=AZURE_DEPLOYMENT_NAME, api_version=AZURE_API_VERSION, temperature=0.7)
manim_model = AzureChatOpenAI(deployment_name=AZURE_DEPLOYMENT_NAME, api_version=AZURE_API_VERSION, temperature=0.0)


# System prompt for script generation
script_system_message = SystemMessage(content="""
You are an advanced AI model with expertise in generating educational scripts.
Your task is to analyze the given topic and generate a well-structured script that explains the topic in simple English.
The script generated will be used to create a video using the Manim library.
Please adhere to the following instructions:
Structure the script into well-formed paragraphs, where each paragraph covers a specific idea or subtopic.
The script should introduce the topic, explain key concepts step by step, and avoid complex words to keep it beginner-friendly.
The final output must be a Python string.
Do not include any LaTeX or Markdown formatting in your output.
Keep the size of the script below 2 paragraph.
**IMPORTANT**: Please separate paragraphs with the marker [SCENE_BREAK].
EXAMPLE:
  Okay, let's break down the Pythagorean Theorem.
**What it is:**

The Pythagorean Theorem is a fundamental relationship in Euclidean geometry among the three sides of a **right triangle**. It states that the square of the length of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the lengths of the other two sides (the legs, or cathetus).
[SCENE_BREAK]
**Formula:**

a² + b² = c²

Where:

*   **a** and **b** are the lengths of the two shorter sides (legs) of the right triangle.
*   **c** is the length of the hypotenuse (the longest side, opposite the right angle).
[SCENE_BREAK]
**Key Points and Concepts:**

*   **Right Triangle:**  The theorem *only* applies to triangles that have one angle that is exactly 90 degrees (a right angle).

*   **Hypotenuse:** The hypotenuse is *always* the longest side of a right triangle, and it's *always* opposite the right angle.

*   **Legs (Cathetus):** The two shorter sides that form the right angle are the legs.

*   **Understanding the Square:** When we say "a²", we mean "a multiplied by itself" (a * a).  Geometrically, a² represents the area of a square with side length 'a'. Similarly for b² and c².

*   **Visual Representation:** Imagine a right triangle. If you draw squares on each of the three sides, the area of the square on the hypotenuse is equal to the *sum* of the areas of the squares on the two legs.
[SCENE_BREAK]
**How to Use It:**

The Pythagorean Theorem is most commonly used in these situations:

1.  **Finding the Hypotenuse:**  If you know the lengths of the two legs (a and b), you can find the length of the hypotenuse (c):
    *   c = √(a² + b²)  (Take the square root of both sides of the original equation)

2.  **Finding a Leg:** If you know the length of the hypotenuse (c) and one leg (either a or b), you can find the length of the other leg:
    *   a = √(c² - b²)
    *   b = √(c² - a²)

3. **Determining if a Triangle is a Right Triangle:** If you know the lengths of all three sides of a triangle, you can use the Pythagorean Theorem to check if it's a right triangle. If a² + b² = c², then it is a right triangle. If not, it's not.
[SCENE_BREAK]
**Example 1: Finding the Hypotenuse**

Let's say you have a right triangle where one leg (a) is 3 units long and the other leg (b) is 4 units long. What is the length of the hypotenuse (c)?

1.  Apply the formula: a² + b² = c²
2.  Substitute the known values: 3² + 4² = c²
3.  Calculate: 9 + 16 = c²
4.  Simplify: 25 = c²
5.  Take the square root of both sides: √25 = c
6.  Solve for c: c = 5

Therefore, the length of the hypotenuse is 5 units.
[SCENE_BREAK]
**Example 2: Finding a Leg**

Let's say you have a right triangle where the hypotenuse (c) is 13 units long and one leg (a) is 5 units long. What is the length of the other leg (b)?

1.  Apply the formula: a² + b² = c²  (We need to rearrange to solve for b)
2.  Rearrange the formula: b² = c² - a²
3.  Substitute the known values: b² = 13² - 5²
4.  Calculate: b² = 169 - 25
5.  Simplify: b² = 144
6.  Take the square root of both sides: √144 = b
7.  Solve for b: b = 12

Therefore, the length of the other leg is 12 units.
[SCENE_BREAK]
**Why it's Important:**

The Pythagorean Theorem is one of the cornerstones of geometry and trigonometry.  It has countless applications in various fields:

*   **Construction:** Calculating lengths and angles for buildings, bridges, etc.
*   **Navigation:** Determining distances and directions.
*   **Engineering:** Designing structures and machines.
*   **Computer Graphics:**  Calculating distances and positions in 3D space.
*   **Physics:** Relating distances and velocities.
[SCENE_BREAK]
**Common Pythagorean Triples:**

A Pythagorean triple is a set of three positive integers (a, b, c) that satisfy the Pythagorean Theorem (a² + b² = c²).  Knowing some common triples can save you time:

*   (3, 4, 5)  (As seen in the example above)
*   (5, 12, 13)
*   (8, 15, 17)
*   (7, 24, 25)
*   (20, 21, 29)

Multiples of these triples are also Pythagorean triples (e.g., (6, 8, 10) is a multiple of (3, 4, 5)).
[SCENE_BREAK]
**In Summary:**

The Pythagorean Theorem is a powerful tool for working with right triangles. Understanding the formula and how to apply it is essential for solving a wide variety of geometric and real-world problems.  Remember to identify the hypotenuse and the legs correctly before applying the formula.
[SCENE_BREAK]
RESPONSE:
""")

# -----------------------------------------------
# Utility Functions
# -----------------------------------------------
def get_audio_length(audio_path):
    """Returns the duration of an audio file in seconds."""
    try:
        clip = AudioFileClip(audio_path)
        duration = clip.duration
        clip.close()
        return duration
    except Exception as e:
        print(f"Error getting audio length: {e}")
        return None

# -----------------------------------------------
# Step 1: Generate Script from Topic
# -----------------------------------------------
def generate_script(topic):
    chat_history = []
    chat_history.append(script_system_message)
    chat_history.append(HumanMessage(content=topic))
    result = script_model.invoke(chat_history)
    return result.content.strip()

# -----------------------------------------------
# Step 2: Split Script into Scenes
# -----------------------------------------------
def split_script(script):
    # Here we split by double newline; adjust if you need a different segmentation.
    paragraphs = script.strip().split("[SCENE_BREAK]")
    return [p.strip() for p in paragraphs if p.strip()]

# -----------------------------------------------
# Step 3: Generate Manim Code for a Scene
# -----------------------------------------------
def generate_manim_code(scene_text, scene_id):
    """Generates Manim animation code with retries if needed."""
    chat_history = []

    max_attempts = 10
    attempt = 0
    error_feedback = "NONE"
    while attempt < max_attempts:
        attempt += 1
        print(f"🔄 Generating Manim code for Scene {scene_id} (Attempt {attempt}/{max_attempts})")
        #Input Script: {scene_text}
        manim_prompt = f"""

        You are an advanced AI model with expertise in generating Manim animations.
        Your task is to analyze the given script and generate **pure Manim Python code** that just represents that content.

        Guidelines:
        - The animation should be clear and simple, avoiding heavy effects.
        - Use the class name as Scene{scene_id} for each scene.
        - Use self.add_sound(\"scene_{scene_id}.mp3\") for voiceover sync and follow it by self.wait(0.5).
        - Use basic Manim functionalities like Create, Transform, FadeIn, FadeOut, Shift, Rotate, etc.
        - Ensure the generated code is correct and complete, ready to be executed.
        - Avoid complex or computationally intensive effects.
        - IMPORTANT: Use self.play(FadeOut(mobject)) to remove objects before the next animation.
        - Use Manim version 2023.7.1.
        - Make sure the text does not go out of the screen.
        - Do not include markdown formatting (like ``` or python) in your code.
        Here is an example code for reference:

        from manim import *

        class Scene1(Scene):
            def construct(self):
                # Add voiceover audio synced with the scene
                self.add_sound("scene_1.mp3")

                # Create and display a title
                title = Text("Understanding the Water Cycle")
                self.play(Write(title))
                self.wait(1)
                self.play(FadeOut(title))

                # Create a sun (represented by a circle)
                sun = Circle(radius=1, color=YELLOW)
                self.play(Create(sun))
                self.wait(1)

                # Transition: Transform the sun into a text label "Evaporation"
                evaporation_label = Text("Evaporation")
                self.play(Transform(sun, evaporation_label))
                self.wait(1)
                self.play(FadeOut(evaporation_label))

                # Create a cloud to represent condensation
                cloud = Ellipse(width=4, height=2, color=GRAY)
                self.play(Create(cloud))
                self.wait(1)
                self.play(FadeOut(cloud))

                # End the scene with a brief pause
                self.wait(1)

        Previous attempt failed due to the following error:
        {error_feedback}
        Please fix this issue in the new code.
        """
        result = manim_model.invoke([SystemMessage(content=manim_prompt), HumanMessage(content=scene_text)])
        generated_code = result.content.strip()
        # If code is wrapped in markdown, extract it:
        pattern = r"```python\n(.*?)\n```"
        match = re.search(pattern, result.content, re.DOTALL)
        if match:
            generated_code = match.group(1)
        if not generated_code.startswith("from manim import *"):
            generated_code = f"from manim import *\n\n{generated_code}"
        # Save the code to file
        filename = f"scene_{scene_id}.py"
        with open(filename, "w") as f:
            f.write(generated_code)
        print(f"✅ Manim code generated for Scene {scene_id} (Attempt {attempt})")
        success, error_feedback = execute_manim(scene_id)
        if success:
            return generated_code
        print(f"❌ Retrying due to error in Manim execution for Scene {scene_id}...")
    print(f"🚨 Max retries reached for Scene {scene_id}. Skipping...")
    return None

# -----------------------------------------------
# Step 4: Generate Voiceover for a Scene
# -----------------------------------------------
def generate_voiceover(scene_text, scene_id):
    # Allow letters, numbers, spaces, and common punctuation
    cleaned_text = re.sub(r"[^a-zA-Z0-9.,!?;:'\"() ]", "", scene_text)
    print(cleaned_text)  # Debugging: Check cleaned text output

    tts = gTTS(text=cleaned_text, lang="en")
    filename = f"scene_{scene_id}.mp3"
    tts.save(filename)
    return filename


# -----------------------------------------------
# Step 5: Execute Manim Code to Generate Scene Video
# -----------------------------------------------
def execute_manim(scene_id):
    """Executes the Manim script for a given scene and returns success status."""
    command = f"manim -qm -v WARNING scene_{scene_id}.py Scene{scene_id}"
    try:
        print(f"🚀 Running Manim for Scene {scene_id}...")
        result = subprocess.run(command, shell=True, capture_output=True, text=True)
        if result.returncode == 0:
            print(f"✅ Manim execution successful for Scene {scene_id}!")
            return True, None
        else:
            error_message = result.stderr.strip()
            print(f"❌ Manim execution failed for Scene {scene_id}!")
            print("🔹 Error Log (last 10 lines):\n", "\n".join(error_message.split('\n')[-10:]))
            return False, error_message
    except Exception as e:
        error_message = str(e)
        print(f"🚨 Critical Error executing Manim for Scene {scene_id}: {error_message}")
        return False, error_message

# -----------------------------------------------
# Step 6: Synchronize Audio and Video for a Scene
# -----------------------------------------------
def synchronize_scene(scene_id, audio_file, video_file):
    """
    Loads the audio and video clips, compares durations, and:
      - If the video is shorter than the audio, freezes the last frame to extend the video.
      - If the audio is shorter than the video, appends silence to the audio.
    Returns the merged clip.
    """
    try:
        video_clip = VideoFileClip(video_file)
        audio_clip = AudioFileClip(audio_file)
    except Exception as e:
        raise Exception(f"Error loading clips for Scene {scene_id}: {e}")

    video_duration = video_clip.duration
    audio_duration = get_audio_length(audio_file)
    max_duration = max(video_duration, audio_duration)
    print(f"Scene {scene_id} durations -- Video: {video_duration:.2f}s, Audio: {audio_duration:.2f}s, Using: {max_duration:.2f}s")

    if video_duration < audio_duration:
        # Extend video by freezing the last frame
        try:
            # freeze effect: freeze last frame for remaining duration
            freeze_effect = freeze(video_clip, t=video_duration - 0.001, freeze_duration=(audio_duration - video_duration))
            extended_video = freeze_effect
            extended_audio = audio_clip
        except Exception as e:
            raise Exception(f"Error freezing video for Scene {scene_id}: {e}")
    elif audio_duration < video_duration:
        # Extend audio by appending silence
        try:
            silence_duration = video_duration - audio_duration
            silence_audio = AudioClip(lambda t: 0, duration=silence_duration, fps=audio_clip.fps)
            extended_audio = concatenate_audioclips([audio_clip, silence_audio])
            extended_video = video_clip
        except Exception as e:
            raise Exception(f"Error extending audio for Scene {scene_id}: {e}")
    else:
        extended_video = video_clip
        extended_audio = audio_clip

    merged_clip = extended_video.set_audio(extended_audio)
    return merged_clip

# -----------------------------------------------
# Step 7: Merge All Scene Videos into Final Video
# -----------------------------------------------
def merge_videos(scene_video_files):
    clips = [VideoFileClip(video) for video in scene_video_files]
    final_video = concatenate_videoclips(clips)
    final_output = "final_video.mp4"
    final_video.write_videofile(final_output, codec="libx264")
    for clip in clips:
        clip.close()
    final_video.close()
    return final_output

# -----------------------------------------------
# Step 8: Cleanup Temporary Files
# -----------------------------------------------
def cleanup_files(scene_count):
    file_patterns = ["scene_{}.py", "scene_{}.mp3", "scene_{}.mp4", "merged_scene_{}.mp4"]
    # Remove Manim's media folder if exists
    shutil.rmtree('/content/media', ignore_errors=True)
    for i in range(1, scene_count + 1):
        for pattern in file_patterns:
            file_path = pattern.format(i)
            if os.path.exists(file_path):
                os.remove(file_path)
                print(f"🗑️ Deleted: {file_path}")
    print("🧹 Cleanup complete! Temporary files removed.")

# -----------------------------------------------
# Main Workflow
# -----------------------------------------------
if __name__ == "__main__":
    topic = input("Enter the topic for video: ")
    # 1. Generate the full script from the topic.
    script = generate_script(topic)
    # 2. Split the script into scenes.
    scenes = split_script(script)
    print(f"Generated {len(scenes)} scenes from script:\n{scenes}")

    merged_scene_files = []  # to store the merged scene video filenames

    # 3. Process each scene.
    for i, scene_text in enumerate(scenes, start=1):
        print(f"\n----- Processing Scene {i} -----")
        # Generate voiceover audio.
        generate_voiceover(scene_text, i)
        print(f"Voiceover for Scene {i} generated.")

        # Generate Manim code and execute to produce the scene video.
        manim_code = generate_manim_code(scene_text, i)
        if not manim_code:
            print(f"Skipping Scene {i} due to code generation issues.")
            continue
        # (The manim_code is already saved in scene_i.py by generate_manim_code.)
        success, _ = execute_manim(i)
        if not success:
            print(f"Skipping Scene {i} due to execution errors.")
            continue
        print(f"Manim video for Scene {i} generated.")

        # Assume Manim output video is at the standard location.
        video_file = f"/content/media/videos/scene_{i}/720p30/Scene{i}.mp4"
        audio_file = f"scene_{i}.mp3"
        try:
            merged_clip = synchronize_scene(i, audio_file, video_file)
            merged_filename = f"merged_scene_{i}.mp4"
            merged_clip.write_videofile(merged_filename)
            merged_scene_files.append(merged_filename)
            merged_clip.close()
        except Exception as e:
            print(e)
            continue

    # 4. Merge all scene videos into one final video.
    if merged_scene_files:
        print("Merging all scene videos into final video...")
        final_video = merge_videos(merged_scene_files)
        print(f"Final video saved as '{final_video}'.")
    else:
        print("No scene videos were generated successfully.")

    # 5. Cleanup temporary files.
    cleanup_files(len(scenes))
    print("Workflow complete!")


Enter the topic for video: Factorial
Generated 2 scenes from script:
['Okay, let\'s talk about factorials. A factorial is a special mathematical function that helps us find the product of all positive integers up to a certain number. We write the factorial of a number n as n! (read as "n factorial"). For example, if we want to calculate 5!, we would multiply 5 by every whole number less than it: 5! = 5 × 4 × 3 × 2 × 1. The result of 5! is 120. It’s important to remember that 0! is defined to be 1, which is a unique case in factorials.', 'Factorials are used in many areas of mathematics, especially in combinations and permutations, which are ways to count how many different arrangements can be made from a set of items. For example, if we have 3 different books and we want to know how many ways we can arrange them on a shelf, we would calculate it as 3! = 6. This means there are 6 different ways to arrange those 3 books. Understanding factorials is essential for solving various problems 

MoviePy - Done.
Moviepy - Writing video merged_scene_1.mp4



t:  85%|████████▌ | 1003/1175 [00:18<00:02, 66.36it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.11/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/media/videos/scene_1/720p30/Scene1.mp4, 2764800 bytes wanted but 0 bytes read,at frame 1005/1175, at time 33.50/39.15 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  86%|████████▌ | 1010/1175 [00:18<00:02, 64.46it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.11/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/media/videos/scene_1/720p30/Scene1.mp4, 2764800 bytes wanted but 0 bytes read,at frame 1009/1175, at time 33.63/39.15 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.

Moviepy - Done !
Moviepy - video ready merged_scene_1.mp4

----- Processing Scene 2 -----
Factorials are used in many areas of mathematics, especially in combinations and permutations, which are ways to count how many different arrangements can be made from a set of items. For example, if we have 3 different books and we want to know how many ways we can arrange them on a shelf, we would calculate it as 3!  6. This means there are 6 different ways to arrange those 3 books. Understanding factorials is essential for solving various problems in probability, statistics, and algebra, making it a fundamental concept in mathematics.
Voiceover for Scene 2 generated.
🔄 Generating Manim code for Scene 2 (Attempt 1/10)
✅ Manim code generated for Scene 2 (Attempt 1)
🚀 Running Manim for Scene 2...
✅ Manim execution successful for Scene 2!
🚀 Running Manim for Scene 2...
✅ Manim execution successful for Scene 2!
Manim video for Scene 2 generated.
Scene 2 durations -- Video: 39.53s, Audio: 39.50s, Usi

MoviePy - Done.
Moviepy - Writing video merged_scene_2.mp4



t:  77%|███████▋  | 915/1186 [00:17<00:04, 61.55it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.11/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/media/videos/scene_2/720p30/Scene2.mp4, 2764800 bytes wanted but 0 bytes read,at frame 915/1186, at time 30.50/39.53 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  78%|███████▊  | 922/1186 [00:18<00:04, 58.28it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.11/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/media/videos/scene_2/720p30/Scene2.mp4, 2764800 bytes wanted but 0 bytes read,at frame 921/1

Moviepy - Done !
Moviepy - video ready merged_scene_2.mp4
Merging all scene videos into final video...
Moviepy - Building video final_video.mp4.
MoviePy - Writing audio in final_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video final_video.mp4



t:  50%|████▉     | 1174/2362 [00:23<00:42, 28.01it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.11/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file merged_scene_1.mp4, 2764800 bytes wanted but 0 bytes read,at frame 1175/1176, at time 39.17/39.17 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready final_video.mp4
Final video saved as 'final_video.mp4'.
🗑️ Deleted: scene_1.py
🗑️ Deleted: scene_1.mp3
🗑️ Deleted: merged_scene_1.mp4
🗑️ Deleted: scene_2.py
🗑️ Deleted: scene_2.mp3
🗑️ Deleted: merged_scene_2.mp4
🧹 Cleanup complete! Temporary files removed.
Workflow complete!


In [19]:
from IPython.display import Video

# Replace 'your_video.mp4' with the uploaded filename
Video("/content/final_video.mp4", embed=True)
